### Import Required Modules

In [ ]:
# ../CBSEData/cbse_gender_names_list.csv

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import pickle

In [2]:
import sys  
sys.path.insert(0, '../../PreProcessing/')

from utils import *
from parse_df import *
from split_name import *

### Train Model on Data

In [37]:
er_df = getERData("../")
er_df = cleanDf(er_df, 'Name')

../../PreProcessing/parse_df.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Name'] = df['Name'].str.lower()
../../PreProcessing/parse_df.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['len']= [len(str(i)) for i in df[col]]


Gender
FEMALE    10405236
MALE      11632598
Name: Name, dtype: int64


In [38]:
er_df = er_df.reset_index(drop=True)

In [5]:
train, test = train_test_split(er_df, test_size=0.3)

In [60]:
print(train.shape, ":", test.shape)

(15563422, 9) : (6670038, 9)


In [ ]:
# train, val, test = splitTrainTestVal(er_df[:500000], 0.8 ,0, 'Name')

In [ ]:
# train1, _, test1 = splitTrainTestVal(er_df[100000:500000], 0.8 ,0, 'Name')

In [11]:
sgd_pipe = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 6), lowercase = True, stop_words = "english")),
    ('tfidf', TfidfTransformer(sublinear_tf = True)),
    ('clf', SGDClassifier(class_weight="balanced",loss="squared_loss",max_iter=10000, random_state=None, validation_fraction = 0.3, average=1000))
])

In [12]:
sgd_pipe.fit(train['Name'].values.astype('str'), train['Gender'].values.astype('str'))

Pipeline(steps=[('vect',
                 CountVectorizer(ngram_range=(1, 6), stop_words='english')),
                ('tfidf', TfidfTransformer(sublinear_tf=True)),
                ('clf',
                 SGDClassifier(average=1000, class_weight='balanced',
                               loss='squared_loss', max_iter=10000,
                               validation_fraction=0.3))])

In [14]:
y_pred=sgd_pipe.predict(test['Name'].values.astype('str'))
# print(classification_report(test['Gender'], y_pred, target_names = ['Male', 'Female']))

In [27]:
y_pred = y_pred.astype(str)

In [54]:
test['Gender'].value_counts()

MALE      3487116
FEMALE    3124440
Name: Gender, dtype: int64

In [59]:
np.unique(y_pred, return_counts = True)

(array(['FEMALE', 'MALE', 'nan'], dtype='<U6'),
 array([3329923, 2903187,  436928]))

In [30]:
print(classification_report(test['Gender'].values.astype('str'), y_pred))

              precision    recall  f1-score   support

      FEMALE       0.82      0.88      0.85   3124440
        MALE       0.93      0.77      0.84   3487116
         nan       0.02      0.18      0.04     58482

    accuracy                           0.82   6670038
   macro avg       0.59      0.61      0.58   6670038
weighted avg       0.87      0.82      0.84   6670038



### Save Model

In [53]:
filename = "../SavedModels/FinalSet/SGDClassifier/ERModel.pkl"
pickle.dump(sgd_pipe, open(filename, 'wb'))